# SVR 사용

In [ ]:
from sklearn.svm import SVR
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# 테슬라 데이터 모두 불러오기 (2020-01-02부터 2022-03-24까지의 데이터만 쓸 예정)
TSLA_data = pd.read_csv("TSLA.csv")
TSLA_data_tail_1 = TSLA_data.tail(562) # (2020-01-02부터 2022-03-24까지의 데이터만 쓸 예정)
TSLA_data_tail_1

In [ ]:
# 3월 24일이라는 실제 값을 빼고 이것을 예측값으로 설정하기 위해 데이터 제거
TSLA_data_tail_2 = TSLA_data_tail_1.head(len(TSLA_data_tail_1)-1)
TSLA_data_tail_2

In [ ]:
# 데이터의 adj_close(조정 종가) 열에 있는 데이터를 NumPy 배열 형태로 저장
TSLA_data_adj_close = TSLA_data_tail_2['Adj Close']
TSLA_data_adj_close = TSLA_data_adj_close.values
TSLA_data_adj_close

In [ ]:
# 테슬라 데이터의 행의 개수만큼 반복 days엔 행의 개수만큼 숫자가 저장
days = []
i = 1
for i in range(len(TSLA_data_tail_2)):
  days.append([int(i+1)])
  i += 1

In [ ]:
# 선형 커널을 사용한 SVR모델을 생성(오차 허용 범위가 1000(C=1000))후 모델을 훈련
lin_svr = SVR(kernel='linear', C=1000)
a = lin_svr.fit(days, TSLA_data_adj_close)

In [ ]:
# 다항 커널을 가진 SVR모델을 생성, 이를 통해 비선형적인 관계를 학습 2차 다항식을 사용하기 위해 degree=2로 설정
poly_svr = SVR(kernel='poly', C=1000, degree=2)
poly_svr.fit(days, TSLA_data_adj_close)

In [ ]:
# 실제 데이터와 선형 커널, 다항 커널을 가진 SVR 모델의 예측값을 비교하여 시각적으로 표현
predicted_prices_linear = lin_svr.predict(days)
predicted_prices_poly = poly_svr.predict(days)

# 실제 데이터와 예측값을 그래프로 표현
plt.plot(days, TSLA_data_adj_close, label='Actual')
plt.plot(days, predicted_prices_linear, label='Linear SVR')
plt.plot(days, predicted_prices_poly, label='Polynomial SVR')
plt.xlabel('Days')
plt.ylabel('Adj Close Price')
plt.legend()
plt.show()

In [ ]:
# 마지막날 실제 주식 종가 : 1013.919983
day = [[len(days)+1]]
print("LInear SVR Predcited Price :", lin_svr.predict(day))
print("Polynimial SVR Predcited Price :", poly_svr.predict(day))

In [ ]:
# 즉, 위 코드는 선형 커널과 다항 커널을 가진 SVR 모델을 사용하여 예측값을 계산하고, 실제 값과의 R² 점수를 계산하여 출력
# R² 점수는 예측 모델의 설명력을 평가하는 지표로, 1에 가까울수록 예측 모델의 성능이 더 우수
# 결론적으로 선형커널을 사용한 SVR모델의 성능이 더 우수
from sklearn.metrics import r2_score

# R^2 점수 계산
lin_r2 = r2_score(TSLA_data_adj_close, predicted_prices_linear)
poly_r2 = r2_score(TSLA_data_adj_close, predicted_prices_poly)

print("lin_R^2 Score:", lin_r2)
print("poly_R^2 Score:", poly_r2)

# 선형회귀 방법

In [ ]:
TSLA_data.tail()

In [ ]:
# 'Date' 열을 날짜 형식으로 변환하고, 해당 데이터의 첫 번째 날짜와 마지막 날짜를 비교하여 총 일수를 계산하고 출력하는 작업을 수행합니다.
TSLA_data['Date'] = pd.to_datetime(TSLA_data['Date'])
print(f'Total days = {(TSLA_data.Date.max()  - TSLA_data.Date.min()).days} days')

In [ ]:
TSLA_data.describe()

In [ ]:
# Plotly의 graph_objs 모듈을 go로 import하는 것을 의미합니다.
import plotly.graph_objs as go
# layout변수에 그래프의 레이아웃 설정
layout = go.Layout(
    title='Stock Prices of Tesla',
    xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Times New Roman',
            size=20,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Price',
        titlefont=dict(
            family='Times New Roman',
            size=20,
            color='#7f7f7f'
        )
    )
)
# x축 데이터로 사용되는 Date 열의 값을 y축 데이터로 사용되는 Adj Close 열의 값을 나타냄
tsla_data = [{'x':TSLA_data['Date'], 'y':TSLA_data['Adj Close']}]
plotTSLA = go.Figure(data=tsla_data, layout=layout)

In [ ]:
#plotTSLA에 저장된 그래프를 인터랙티브하게 출력합니다. 인터랙티브한 출력은 웹 브라우저에서 그래프를 상호작용적으로 탐색하고 확대/축소, 툴팁, 그래프 내 데이터 포인트의 값을 확인하는 등의 작업이 가능
from plotly.offline import iplot
iplot(plotTSLA)

In [ ]:
# 데이터를 훈련 세트와 테스트 세트로 분할하기 위해 train_test_split() 함수를 import
from sklearn.model_selection import train_test_split

# 데이터 전처리를 위해 import
from sklearn.preprocessing import MinMaxScaler # 데이터를 최소값과 최대값 사이의 범위로 변환
from sklearn.preprocessing import StandardScaler # 데이터를 평균과 표준편차를 기준으로 표준화

#모데을 평가를 위해 평균제곱 오차와  R² 스코어를 import
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

In [ ]:
# 'TSLA_data'의 인덱스를 2차원 배열 형태로 변환하여 'XT' 변수에 저장, 1개의 열을 가지는 2차원 배열
XT = np.array(TSLA_data.index).reshape(-1,1)
YT = TSLA_data['Adj Close']
# test_size=0.3은 테스트 세트의 비율을 0.3로 설정
XT_train, XT_test, YT_train, YT_test = train_test_split(XT, YT, test_size=0.3, random_state=42)

In [ ]:
# 선형 회귀 모델을 사용하기 위해 import
from sklearn.linear_model import LinearRegression

In [ ]:
# 모델 생성후 훈련
lmT = LinearRegression()
lmT.fit(XT_train, YT_train)

In [ ]:
lmT.predict(XT_train).T

In [ ]:
trace0T = go.Scatter(
    x = XT_train.T[0], # .T 행과 열을 바꾸는 연산 // 2차원 배열로 구성되어 있던 XT_train이 1차원 배열로 변환. T[0]을 사용하면 전치된 배열의 첫 번째 열에 해당하는 값들만 가져오기 때문
    y = YT_train,
    mode = 'markers',
    name = 'Actual'
)
trace1T = go.Scatter(
    x = XT_train.T[0], # .T 행과 열을 바꾸는 연산 // 2차원 배열로 구성되어 있던 XT_train이 1차원 배열로 변환. T[0]을 사용하면 전치된 배열의 첫 번째 열에 해당하는 값들만 가져오기 때문
    y = lmT.predict(XT_train),
    mode = 'lines',
    name = 'Predicted'
)
tesla_data = [trace0T,trace1T] # 실제 값과 예측 값에 해당하는 값들을 해당하는 트레이스 객체들을 리스트로  묶어 저장
layout.xaxis.title.text = 'Day'
plot2 = go.Figure(data=tesla_data, layout=layout) # Figure클래스를 사용하여 그래프 생성

In [ ]:
# 그래프로 출력
iplot(plot2)

In [ ]:
scoresT = f'''
{'Metric'.ljust(10)}{'Train'.center(20)}{'Test'.center(20)}
{'r2_score'.ljust(10)}{r2_score(YT_train, lmT.predict(XT_train))}\t{r2_score(YT_test, lmT.predict(XT_test))}
{'MSE'.ljust(10)}{mse(YT_train, lmT.predict(XT_train))}\t{mse(YT_test, lmT.predict(XT_test))}
'''
#1행 표의 헤더 부분을 정의 Metric는 10칸으로 왼쪽 정렬, Train과 Test는 20칸으로 가운데 정렬
#2행 r2_score'는 10칸으로 왼쪽 정렬, 계산된 훈련 데이터와 테스트 데이터에 대한 r2_score 값이 포맷팅되어 출력
#3행'MSE'는 10칸으로 왼쪽 정렬, 계산된 훈련 데이터와 테스트 데이터에 대한 mean_squared_error 값이 포맷팅되어 출력

print(scoresT)

# 훈련데이터와 테스트 데이터 모두에 대해 비슷한 수준의 성능을 보이고 있지만 예측값과 실제값 사이에는 큰 차이가 있음을 나타냄.

방법 1은 데이터의 훈련 및 테스트 세트 분할 없이 전체 데이터를 사용하여 모델을 훈련시키는 방법입니다. 따라서, days와 TSLA_data_adj_close 데이터를 모두 사용하여 단일 모델을 훈련시키고, 훈련된 모델은 a 변수에 저장됩니다.

방법 2는 데이터를 훈련 및 테스트 세트로 분할한 후 훈련 데이터를 사용하여 모델을 훈련시키는 방법입니다. train_test_split() 함수를 사용하여 데이터를 분할한 다음, X_train과 y_train을 사용하여 모델을 훈련시킵니다. 이를 통해 모델의 성능을 평가하기 위해 테스트 데이터를 사용할 수 있습니다.

따라서, 방법 1은 전체 데이터를 사용하여 훈련하는 단일 모델을 생성하고, 방법 2는 데이터를 훈련 및 테스트 세트로 분할한 후 모델을 훈련하는 방법입니다. 선택하는 방법은 데이터의 성격과 목표에 따라 다를 수 있습니다.